In [8]:
catelog = {
  "categorys": (
#       "BODIES & SPARKDRÄKTER","BADBYXOR","BADDRÄKTER","UV-KLÄDER","byxor","jeans","shorts","skirts","dresses","costumes","pyjamas","kardagins",
#                 "tops","sweaters","underpants","leggings","socks","panties","jackets",
      "overall","rainwear"),

   "sub_urls": (
#     "https://cdon.se/klader/barn/bodies-sparkdrakter/",
#                "https://cdon.se/klader/barn/badklader/badbyxor/",
#                "https://cdon.se/klader/barn/badklader/baddrakter/",
#                "https://cdon.se/klader/barn/badklader/uv-klader/","https://cdon.se/klader/barn/byxor/byxor/",
#                "https://cdon.se/klader/barn/byxor/jeans/","https://cdon.se/klader/barn/byxor/shorts/",
#                "https://cdon.se/klader/barn/klanningar-kjolar/kjolar/",
#                "https://cdon.se/klader/barn/klanningar-kjolar/klanningar/","https://cdon.se/klader/barn/kostymer/",
#                "https://cdon.se/klader/barn/pyjamas/","https://cdon.se/klader/barn/trojor-koftor/koftor/",
#                "https://cdon.se/klader/barn/trojor-koftor/t-shirt-toppar/","https://cdon.se/klader/barn/trojor-koftor/trojor/",
#                "https://cdon.se/klader/barn/underklader/kalsonger/","https://cdon.se/klader/barn/underklader/strumpbyxor/",
#                "https://cdon.se/klader/barn/underklader/strumpbyxor/","https://cdon.se/klader/barn/underklader/strumpor/",
#                "https://cdon.se/klader/barn/underklader/trosor/","https://cdon.se/klader/barn/ytterplagg/jackor/",
               "https://cdon.se/klader/barn/ytterplagg/overall/","https://cdon.se/klader/barn/ytterplagg/regnklader/"
              )
  }

In [2]:
catelog["categorys"][0]

'BODIES & SPARKDRÄKTER'

In [7]:
len(catelog["categorys"])

21

In [3]:
import csv


f = open("cdon.csv", "w", encoding='utf-8')
writer = csv.DictWriter(f, fieldnames=["name","brand","gender","category","size","price","color","url","img_url","article_no"])
writer.writeheader()
f.close()

import pandas as pd
df=pd.read_csv("cdon.csv", encoding='utf-8')
df.head()

,name,brand,gender,category,size,price,color,url,img_url,article_no


In [4]:
!mkdir cdon

mkdir: cannot create directory ‘cdon’: File exists


In [9]:
import requests
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.62 Safari/537.36'}
from bs4 import BeautifulSoup
import urllib.request
import csv

def scrapper(brand='cdon',color='None',gender='unisex'):
    for c in range(len(catelog['sub_urls'])):
        urls=catelog['sub_urls'][c]
        category=catelog['categorys'][c]
        p=requests.get(urls, headers=headers)
        html_doc=p.content.decode()
        soup = BeautifulSoup(html_doc, 'html.parser')
        
        pages=soup.find("ul",attrs={"class":"pagination"})
        page=[urls]
        try:
            for p in pages.find_all("a"):
                if urls+p["href"] not in page:
                    page.append(urls+p["href"])
        except:
            pass
        
        for ip in page:
            print(ip)
            q=requests.get(ip, headers=headers)
            html_doc2=q.content.decode()
            soup2 = BeautifulSoup(html_doc2, 'html.parser')
        

            main_products=soup2.find("ul",attrs={"class":"product-list medium column-count-4 hoverable"})
            products=main_products.find_all("article", attrs={ "class" :"product product-info"})
            print(len(products))
            
            for p in products:
                
                try:
                    article_no=p.select_one('.product-id')['value'] #css selector
                    name=p.select_one('.full-title')['value']
                    url='https://cdon.se'+p.a['href']
                    price=soup.find("div", attrs={ "class" :"product-price-wrapper"}).text.strip()
    #                 print(article_no,name,url,price)

                    s=requests.get(url, headers=headers)
                    html_doc4=s.content.decode()
                    soup4 = BeautifulSoup(html_doc4, 'html.parser')
                    description=soup4.find("div",attrs={"id":"facts-data-tab"})
                    tr=description.find_all("tr")
                    for t in tr:
                        if t.th.text=='Varumärke':
                            brand=t.td.text
                        if t.th.text=='Färg':
                            color=t.td.text
                        if t.th.text=='Kön':
                            gender=t.td.text
                    img_url=soup4.find("div",attrs={"class":"product-image-container"}).find("a")['href']
                    #print(brand,color,img_url)


                    ac='cdon'+'/'+article_no+str('.jpg')
                    opener = urllib.request.URLopener()
                    opener.addheader('User-Agent', 'whatever')
                    filename, headers2 = opener.retrieve(img_url, ac)
                    c=0
                    for ij in soup4.find("select",attrs={"class":"variation-items-select-box"}).find_all("option"):
                        if c>0:
                            size=ij.text.strip()
    #                         print(size)
                            with open("cdon.csv", "a", encoding='utf-8') as f:
                                writer = csv.writer(f)
                                writer.writerow([name,brand,gender,category,size,price,color,url,img_url,article_no])
                        c+=1

    #                     print(name,price,color,composition,article_no,img_url,size)
                except:
                    continue

                

In [10]:
scrapper()

https://cdon.se/klader/barn/ytterplagg/overall/
32


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: URLopener style of invoking requests is deprecated. Use newer urlopen functions/methods
  """Entry point for launching an IPython kernel.


https://cdon.se/klader/barn/ytterplagg/overall/?blockid=699154&page=2
15
https://cdon.se/klader/barn/ytterplagg/regnklader/
29
https://cdon.se/klader/barn/ytterplagg/regnklader/?blockid=699134&page=2
28
https://cdon.se/klader/barn/ytterplagg/regnklader/?blockid=699134&page=3
25


In [ ]:
urls=catelog['sub_urls'][0]
category=catelog['categorys'][0]

p=requests.get(urls, headers=headers)
html_doc=p.content.decode()
soup = BeautifulSoup(html_doc, 'html.parser')

main_products=soup.find("ul",attrs={"class":"product-list medium column-count-4 hoverable"})
products=main_products.find_all("article", attrs={ "class" :"product product-info"})
print(len(products))
for p in products:
    article_no=p.select_one('.product-id')['value'] #css selector
    name=p.select_one('.full-title')['value']
    url='https://cdon.se'+p.a['href']
    price=soup.find("div", attrs={ "class" :"product-price-wrapper"}).text.strip()
    print(article_no,name,url,price)
    
    s=requests.get(url, headers=headers)
    html_doc4=s.content.decode()
    soup4 = BeautifulSoup(html_doc4, 'html.parser')
    description=soup4.find("div",attrs={"id":"facts-data-tab"})
    tr=description.find_all("tr")
    for t in tr:
        if t.th.text=='Varumärke':
            brand=t.td.text
        if t.th.text=='Färg':
            color=t.td.text
        if t.th.text=='Kön':
            sex=t.td.text
    img_url=soup4.find("div",attrs={"class":"product-image-container"}).find("a")['href']
    print(brand,color,img_url,sex)
    
    

    break

In [ ]:
main_products.find("article", attrs={ "class" :"product product-info"})

In [ ]:
s=requests.get(url, headers=headers)
html_doc4=s.content.decode()
soup4 = BeautifulSoup(html_doc4, 'html.parser')
description=soup4.find("div",attrs={"id":"facts-data-tab"})
tr=description.find_all("tr")
for t in tr:
    if t.th.text=='Varumärke':
        brand=t.td.text
    if t.th.text=='Färg':
        color=t.td.text
img_url=soup4.find("div",attrs={"class":"product-image-container"}).find("a")['href']
print(brand,color,img_url)


In [ ]:
url='https://cdon.se/klader/babybyxor-gra-denim-p42480575'

In [ ]:
c=0
for ij in soup4.find("select",attrs={"class":"variation-items-select-box"}).find_all("option"):
    if c>0:
        size=ij.text.strip()
        print(size)
    c+=1

In [ ]:
type(size)

In [ ]:
description

In [ ]:
pages=soup.find("ul",attrs={"class":"pagination"})
page=[urls]
for p in pages.find_all("a"):
    if urls+p["href"] not in page:
        page.append(urls+p["href"])
        print(page)
    

In [ ]:
page